In [ ]:
import numpy as np
import pandas as pd 

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from catboost import Pool, CatBoostClassifier, cv

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')
sample = pd.read_csv('../input/tabular-playground-series-apr-2021/sample_submission.csv')

In [ ]:
train

In [ ]:
test

In [ ]:
train.drop(['PassengerId','Name'], axis = 1, inplace = True)
test.drop(['PassengerId','Name'], axis = 1, inplace = True)

In [ ]:
train.fillna(-999,inplace=True)
test.fillna(-999,inplace=True)

In [ ]:
target = train['Survived']

train.drop('Survived', axis = 1, inplace = True)

In [ ]:
#X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.15, random_state=1, stratify=target)

In [ ]:
#cat_features_index = np.where(train.dtypes == object)[0]
cat_features_index = [0,1,5,7,8]

In [ ]:
cb = CatBoostClassifier(random_state = 1,loss_function="Logloss",
                        verbose=200, eval_metric='Accuracy', thread_count=-1, task_type='GPU', iterations=1500)

In [ ]:
cb.fit(train,target, plot=True,# use_best_model=True,eval_set=(X_val, y_val),
       cat_features=cat_features_index)

In [ ]:
train_pred = cb.predict(train, thread_count=-1)
y_pred = cb.predict(test, thread_count=-1)
#val_pred = cb.predict(X_val, thread_count=-1)

In [ ]:
print("Train Accuracy  ",accuracy_score(target,train_pred))
#print("Val Accuracy    ",accuracy_score(y_val,val_pred))

In [ ]:
feat_importance = [col for col in zip(train.columns,cb.get_feature_importance() )]
feat_importance_df = pd.DataFrame(feat_importance, columns=['Feature','Feature Importance'])
feat_importance_df = feat_importance_df.sort_values('Feature Importance', ascending = False)
feat_importance_df

In [ ]:
sample['Survived'] = y_pred
sample

In [ ]:
sample.to_csv("cb_baseline.csv",index=False)